<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_base_help_class_no_pretrain_multi_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import time
start_time = time()

In [2]:
!pip install transformers[torch]
!pip install datasets
!pip install scikit-learn
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from sklearn.metrics import accuracy_score,  f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [14]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset"
dataset = load_dataset(dataset_name)
dataset

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [15]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
from transformers import TrainingArguments

model_name = "micro_base_help_class_no_pre"

training_args = TrainingArguments(
    output_dir=model_name,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0001,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10, # NEED TO TRAIN FOR REAL TEST
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    save_total_limit=1, # Saves latest 2 checkpoints
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    load_best_model_at_end=True, # Set to false - we want the last trained model like the paper
    metric_for_best_model="f1_macro", # Is default
    # torch_compile=True,  # Much Faster
    logging_strategy="steps", # Is default
    logging_steps=100, # Logs training progress
    seed=0,
)

In [17]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback with patience
early_stopping = EarlyStoppingCallback(early_stopping_patience=3) # from paper

In [18]:
from transformers import Trainer

import gc


In [19]:
from transformers import AutoConfig

#Label Settings
id2label = {0: "unhelpful", 1: "helpful"}
label2id = {"unhelpful": 0, "helpful": 1}

# Set Classifier Settings
classification_config = AutoConfig.from_pretrained('roberta-base')
classification_config.classifier_dropout = 0.1 # From Paper
classification_config.num_of_labels = 2
classification_config.id2label=id2label
classification_config.label2id=label2id,
classification_config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "unhelpful",
    "1": "helpful"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": [
    {
      "helpful": 1,
      "unhelpful": 0
    }
  ],
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_of_labels": 2,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.36.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [20]:
from transformers import AutoModelForSequenceClassification

In [21]:
all_train_results = []
all_eval_results = []

In [ ]:
for seed in range(5):
  print(f"Seed: {seed}")
  print("=========================================")
  training_args.seed = seed

  model = AutoModelForSequenceClassification.from_pretrained(
      "roberta-base",  config=classification_config
  )

  training_args.output_dir = f"{model_name}_seed_{seed}"

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=dataset["train"],
      eval_dataset=dataset["dev"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks=[early_stopping]
  )

  gc.collect()

  results = trainer.train()
  print(results)
  all_train_results.append(results)

  eval_results = trainer.evaluate(dataset["test"])
  print(eval_results)
  all_eval_results.append(eval_results)

  trainer.push_to_hub()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Seed: 0


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
for result in all_train_results:
  print(result)

for result in all_eval_results:
  print(result)

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Script Runtime: {total_time / 60:.0f}  minutes OR {total_time:.0f}  seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()
